In [28]:
import os
import platform
import psutil
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Inicializar o app
app = dash.Dash(__name__)

# Funções para coletar informações do sistema
def get_system_info():
    return {
        "Sistema Operacional": platform.system(),
        "Versão": platform.version(),
        "Arquitetura": platform.architecture()[0],
        "Processador": platform.processor(),
        "Nome do Host": platform.node()
    }

def get_hardware_info():
    mem = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    cpu_percent = psutil.cpu_percent(interval=None)
    cpu_idle = 100 - cpu_percent
    return {
        "CPU Uso (%)": cpu_percent,
        "CPU Ocioso (%)": cpu_idle,
        "Memória Total (GB)": round(mem.total / 1e9, 2),
        "Memória Usada (GB)": round(mem.used / 1e9, 2),
        "Espaço Total Disco (GB)": round(disk.total / 1e9, 2),
        "Espaço Usado Disco (GB)": round(disk.used / 1e9, 2),
        "Total de Processos": len(psutil.pids())
    }

def get_process_info():
    return [
        {
            "PID": proc.info["pid"],
            "Nome": proc.info["name"],
            "Usuário": proc.info["username"],
            "CPU (%)": proc.info["cpu_percent"],
            "Memória (%)": proc.info["memory_percent"],
        }
        for proc in psutil.process_iter(["pid", "name", "cpu_percent", "memory_percent", "username"])
    ]

def get_process_threads(pid):
    try:
        proc = psutil.Process(pid)
        threads = proc.threads()
        return [{"ID": thread.id, "Tempo (s)": thread.user_time + thread.system_time} for thread in threads]
    except psutil.NoSuchProcess:
        return []

# Layout do Dashboard
app.layout = html.Div([
    html.H1("Dashboard do Sistema Operacional", style={'text-align': 'center', 'margin-bottom': '40px'}),

    # Informações Globais
    html.Div([
        html.H3("Informações Globais", style={'color': '#66ff66'}),
        html.Div([
            html.Ul(id='system-info'),
            html.Ul(id='hardware-info'),
        ], style={'background-color': '#2e2e2e', 'padding': '20px', 'border-radius': '10px'})
    ], style={'margin-bottom': '40px'}),

    # Gráfico de uso da CPU
    dcc.Graph(id='cpu-usage', style={'margin': '20px', 'border': '1px solid #444', 'border-radius': '10px'}),

    # Tabela de Processos
    html.Div([
        html.H3("Processos em Execução", style={'color': '#66ff66'}),
        dcc.Dropdown(id='process-select', placeholder="Selecione um processo para detalhes"),
        html.Div([html.Table(id='process-table')])
    ], style={'margin-bottom': '40px'}),

    # Detalhes de Processo em Abas
    html.Div([
        dcc.Tabs(id='process-tabs', value='info', children=[
            dcc.Tab(label='Informações Detalhadas', value='info'),
            dcc.Tab(label='Threads do Processo', value='threads'),
        ]),
        html.Div(id='process-tab-content', style={'background-color': '#2e2e2e', 'padding': '20px', 'border-radius': '10px'})
    ]),

    # Intervalo de Atualização
    dcc.Interval(id='update-interval', interval=5000, n_intervals=0)
], style={
    'background-color': '#1e1e1e',
    'color': 'white',
    'padding': '20px',
    'font-family': 'Arial, sans-serif'
})

# Callbacks
@app.callback(
    [Output('system-info', 'children'),
     Output('hardware-info', 'children'),
     Output('cpu-usage', 'figure'),
     Output('process-table', 'children'),
     Output('process-select', 'options')],
    [Input('update-interval', 'n_intervals')]
)
def update_dashboard(_):
    # Informações do sistema
    system_info = get_system_info()
    system_info_list = [html.Li(f"{key}: {value}") for key, value in system_info.items()]

    # Informações de hardware
    hardware_info = get_hardware_info()
    hardware_info_list = [html.Li(f"{key}: {value}") for key, value in hardware_info.items()]

    # Gráfico de uso de CPU
    figure = {
        'data': [{'x': list(range(10)), 'y': [psutil.cpu_percent() for _ in range(10)], 'type': 'line', 'name': 'CPU Usage'}],
        'layout': {'title': 'Uso da CPU (%)', 'plot_bgcolor': '#1e1e1e', 'paper_bgcolor': '#1e1e1e'}
    }

    # Processos em execução
    processes = get_process_info()
    table_header = html.Tr([html.Th(col) for col in ["PID", "Nome", "Usuário", "CPU (%)", "Memória (%)"]])
    table_rows = [
        html.Tr([html.Td(proc[col]) for col in ["PID", "Nome", "Usuário", "CPU (%)", "Memória (%)"]]) for proc in processes
    ]
    process_table = [table_header] + table_rows

    # Opções para o dropdown de seleção de processos
    process_options = [{'label': f"{proc['Nome']} (PID: {proc['PID']})", 'value': proc['PID']} for proc in processes]

    return system_info_list, hardware_info_list, figure, process_table, process_options

@app.callback(
    Output('process-tab-content', 'children'),
    [Input('process-select', 'value'),
     Input('process-tabs', 'value')]
)
def update_process_tabs(selected_pid, tab_value):
    if not selected_pid:
        return html.P("Selecione um processo para ver os detalhes.")

    try:
        proc = psutil.Process(selected_pid)

        if tab_value == 'info':
            details = {
                "PID": proc.pid,
                "Nome": proc.name(),
                "Usuário": proc.username(),
                "CPU (%)": proc.cpu_percent(interval=0.1),
                "Memória (%)": proc.memory_percent(),
                "Status": proc.status(),
                "Tempo de Execução": proc.create_time(),
            }
            return html.Ul([html.Li(f"{key}: {value}") for key, value in details.items()])

        elif tab_value == 'threads':
            threads = get_process_threads(selected_pid)
            thread_header = html.Tr([html.Th("ID da Thread"), html.Th("Tempo (s)")])
            thread_rows = [html.Tr([html.Td(thread["ID"]), html.Td(thread["Tempo (s)"])]) for thread in threads]
            return html.Table([thread_header] + thread_rows)
    except psutil.NoSuchProcess:
        return html.P("O processo selecionado não está mais ativo.")

# Rodar o servidor
if __name__ == '__main__':
    app.run_server(debug=True)